In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import ShortType
from delta.tables import DeltaTable

CATALOGO_ORIGEM = "v_credit"
SCHEMA_ORIGEM = "bronze"
TABELA_ORIGEM = "base_atendentes"

CATALOGO_DESTINO = "v_credit"
SCHEMA_DESTINO = "silver"
TABELA_DESTINO = "tb_atendente"

nome_tabela_origem = f"{CATALOGO_ORIGEM}.{SCHEMA_ORIGEM}.{TABELA_ORIGEM}"
nome_tabela_destino = f"{CATALOGO_DESTINO}.{SCHEMA_DESTINO}.{TABELA_DESTINO}"

In [ ]:
df_bronze = spark.read.table(nome_tabela_origem)

print("Amostra dos dados Bronze:")
df_bronze.display()

print("\n Schema da tabela:")
df_bronze.printSchema()

print("\n Análise de valores nulos:")
nulos_df = df_bronze.select([
    F.count(F.when(F.col(c).isNull(), c)).alias(c)
    for c in df_bronze.columns
])
nulos_df.display()

print("\n Distribuição de níveis de atendimento:")
df_bronze.groupBy("nivel_atendimento").count().orderBy("nivel_atendimento").display()

In [ ]:
df_bronze = spark.read.table(nome_tabela_origem)

df_silver = df_bronze \
    .select(
        F.col("id_atendente").alias("cd_atendente"),
        F.trim(F.col("nome_atendente")).alias("nm_atendente"),
        F.col("nivel_atendimento").cast(ShortType()).alias("nu_nivel"),
        F.current_timestamp().alias("dt_ingestion"),
        F.col("origem").alias("dc_origem")
    ) \
    .filter(F.col("id_atendente").isNotNull()) \
    .dropDuplicates(["cd_atendente"])

delta_table = DeltaTable.forName(spark, nome_tabela_destino)

delta_table.alias("destino").merge(
    df_silver.alias("origem"),
    "destino.cd_atendente = origem.cd_atendente"
).whenMatchedUpdateAll(
).whenNotMatchedInsertAll(
).execute()

print(f"✅ Tabela {nome_tabela_destino} carregada com sucesso")
